In [2]:
from qiskit import *
from qiskit.tools.visualization import plot_histogram
import array as arr

def good_linapx(_l_x, _l_fx):
	fx = arr.array('d', [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7])
	c1 = 0
	c2 = 0
	for i in range(16):
		r1 = 0
		r2 = 0
		l_x = _l_x
		l_fx = _l_fx
		x = i
		f_x = int(fx[i])
		while x > 0:
			r1 ^= (x & 1) & (l_x & 1)
			x >>= 1
			l_x >>= 1
		while f_x > 0:
			r2 ^= (f_x & 1) & (l_fx & 1)
			f_x >>= 1
			l_fx >>= 1
		if r1 == r2:
			c1 += 1
		else:
			c2 += 1
	return (c1 - c2)

In [3]:
qr = QuantumRegister(8)
circuit = QuantumCircuit(qr)

count_trivial = 0
count_bad_linapx = 0

circuit.h(qr[0])
circuit.h(qr[1])
circuit.h(qr[2])
circuit.h(qr[3])

circuit.x(qr[4])
circuit.rcccx(qr[0], qr[1], qr[2], qr[4])
circuit.rccx(qr[0], qr[1], qr[4])
circuit.cx(qr[0], qr[4])
circuit.rcccx(qr[1], qr[2], qr[3], qr[4])
circuit.rccx(qr[1], qr[2], qr[4])
circuit.cx(qr[1], qr[4])
circuit.cx(qr[3], qr[4])

circuit.x(qr[5])
circuit.rcccx(qr[0], qr[2], qr[3], qr[5])
circuit.rccx(qr[0], qr[2], qr[5])
circuit.cx(qr[0], qr[5])
circuit.rccx(qr[1], qr[3], qr[5])
circuit.cx(qr[1], qr[5])
circuit.rccx(qr[2], qr[3], qr[5])

circuit.x(qr[6])
circuit.rcccx(qr[0], qr[1], qr[2], qr[6])
circuit.rcccx(qr[0], qr[1], qr[3], qr[6])
circuit.rccx(qr[0], qr[1], qr[6])
circuit.rccx(qr[0], qr[2], qr[6])
circuit.rccx(qr[0], qr[3], qr[6])
circuit.rccx(qr[1], qr[2], qr[6])
circuit.rccx(qr[1], qr[3], qr[6])
circuit.rccx(qr[2], qr[3], qr[6])
circuit.cx(qr[2], qr[6])
circuit.cx(qr[3], qr[6])

circuit.rcccx(qr[0], qr[2], qr[3], qr[7])
circuit.rccx(qr[0], qr[3], qr[7])
circuit.cx(qr[0], qr[7])
circuit.rccx(qr[1], qr[3], qr[7])
circuit.cx(qr[2], qr[7])

circuit.h(qr[0])
circuit.h(qr[1])
circuit.h(qr[2])
circuit.h(qr[3])
circuit.h(qr[4])
circuit.h(qr[5])
circuit.h(qr[6])
circuit.h(qr[7])

circuit.measure_all()

In [5]:
from qiskit import IBMQ
IBMQ.save_account('5224513159a81052289555d20e4d70dc6369caac4940fe9edb7f15d54af90dce7260ca75b22cb652ae6ede03a3b50dfc2a59aa57c0246e7cff2ecf0fb1ba48f4')
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')
qcomp = provider.get_backend('ibmq_16_melbourne')

configrc.store_credentials:WARNING:2020-10-10 17:25:53,045: Credentials already present. Set overwrite=True to overwrite.


In [8]:
job = execute(circuit, backend = qcomp)
from qiskit.tools.monitor import job_monitor
job_monitor(job)
counts = job.result().get_counts()

Job Status: job has successfully run


In [9]:
try:
	count_trivial = counts['{0:08b}'.format(0)]
except:
	print('Trivial solution occurred 0 time')
if count_trivial != 0:
	print('Trivial solution occur ' + str(count_trivial) + ' times')

count_bad_linapx = 0
for i in range(256):
	if '{0:08b}'.format(i) in counts:
		occurs = counts['{0:08b}'.format(i)]
		if occurs > 0 and good_linapx((i >> 4), (i & 15)) == 0:
			count_bad_linapx += occurs

print('Bad linear approximation occur ' + str(count_bad_linapx) + ' times')
print(counts)

Trivial solution occur 4 times
Bad linear approximation occur 494 times
{'00000000': 4, '00001010': 8, '01000001': 9, '01010010': 4, '10000000': 6, '00000111': 4, '01101101': 5, '00010110': 3, '11100011': 3, '01011011': 2, '01100011': 4, '01101111': 7, '00101001': 5, '11011101': 2, '10001100': 10, '01110101': 1, '10111111': 1, '10100011': 8, '00100111': 2, '01100001': 3, '10111100': 1, '00010100': 2, '01001100': 9, '11101000': 5, '10011010': 1, '01001011': 12, '10000011': 6, '01111100': 4, '00001001': 7, '10011101': 1, '11101001': 3, '00011110': 1, '00111000': 1, '10001011': 2, '01111001': 3, '00111101': 3, '10101110': 1, '11000011': 10, '10101000': 3, '01110001': 1, '01110111': 1, '11011011': 4, '10010111': 1, '10101011': 4, '01101100': 6, '11000110': 5, '00011101': 1, '01101110': 5, '11111010': 2, '01001010': 8, '00100101': 6, '00110101': 3, '01100010': 7, '00001111': 6, '10001000': 7, '00011100': 3, '01000100': 9, '01111110': 1, '01100111': 3, '01010111': 3, '01111111': 3, '11000010